# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-3961000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-1887000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-1562000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, True

# you sit West and hold:
hand = 'AKQJ95.J54..K953'

auction = ["PAD_START", "2D", "PASS"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 3, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '2S', 'insta_score': 0.109, 'expected_score': 240, 'adjustment': 5},
 {'call': 'PASS',
  'insta_score': 0.883,
  'expected_score': -20,
  'adjustment': 44}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [3]:
### Samples consistent with the auction so far

for i in range(min(10,len(bid.samples))):
    print(bid.samples[i])


Tx.AK8xx.Axx.Qxx x.Txx.KQJ9xx.JT8 8xxx.Q9.T8xx.Axx AKQJ9x.Jxx..K9xx 0.79813 
 2D-P-P-2H-P-P-2S-P-P-P (7.7)  
 2D-P-2S-3H-P-P-3S-P-P-P (7.7) 
x.Q98xx.AT8.JTxx 8xx.Ax.Q9xxxx.Ax Txx.KTx.KJxx.Q8x AKQJ9x.Jxx..K9xx 0.84467 
 2D-P-P-P (7.2)  
 2D-P-2S-P-4S-P-P-P (11.2) 
8x.AKxx.KJ8x.Q8x xx.9.AT9xxx.AJTx Txx.QT8xx.Qxx.xx AKQJ9x.Jxx..K9xx 0.98515 
 2D-P-P-P (8.9)  
 2D-P-2S-P-3C-P-3S-P-4S-P-P-P (11.6) 
Txx.KQ9x.Kxx.QJx xx.Txx.AQJ8xx.Tx 8x.A8x.T9xx.A8xx AKQJ9x.Jxx..K9xx 0.89768 
 2D-P-P-P (7.1)  
 2D-P-2S-P-2N-P-3S-P-4S-P-P-P (8.9) 
xx.AQ98x.QTxx.Ax x.KT.A9xxxx.QJT8 T8xx.xxx.KJ8.xxx AKQJ9x.Jxx..K9xx 0.93333 
 2D-P-P-2H-P-P-2S-P-P-P (10.1)  
 2D-P-2S-P-3C-P-3S-P-4S-P-P-P (10.1) 
8x.KT8x.Kx.QJTxx x.9xx.AJ98xx.A8x Txxx.AQx.QTxxx.x AKQJ9x.Jxx..K9xx 0.98517 
 2D-P-P-P (6.8)  
 2D-P-2S-P-2N-P-3S-P-4S-P-P-P (8.9) 
8xx.AQT9x.Kx.Jxx xx.8x.AJT98x.A8x Tx.Kxx.Qxxxx.QTx AKQJ9x.Jxx..K9xx 0.98670 
 2D-P-P-2H-P-P-2S-P-P-3H-P-P-P (6.0)  
 2D-P-2S-P-2N-P-3S-P-4S-P-P-P (10.3) 
x.QT8x.AKx.QJTxx 8x.AKx.Q9xxxx.8x Txx

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [4]:
# we got some samples above. if we want more, we can always get more

auction = ["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"]
sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands_auction(auction)

for i in range(min(10,sample_hands.shape[0])):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])} {sorted_score[i]:.2f}')

AKQJ9x.Jxx..K9xx xx.AQ8.JTxxx.Txx .T9xx.AKQxx.A8xx T8xxx.Kxx.98x.QJ 0.99
AKQJ9x.Jxx..K9xx 8xx.KTx.QJxx.Qxx .Q8xx.AK8xx.AJxx Txxx.A9x.T9xx.T8 0.99
AKQJ9x.Jxx..K9xx Txxx.T8x.Q8xx.Jx x.Kxxx.AKTx.A8xx 8x.AQ9.J9xxx.QTx 0.98
AKQJ9x.Jxx..K9xx xx.8x.KJ9xx.Jxxx 8.KT9x.QT8x.AQT8 Txxx.AQxx.Axxx.x 0.98
AKQJ9x.Jxx..K9xx T8x.A8xx.Txxx.Tx x.KQTx.A98x.AJ8x xxx.9x.KQJxx.Qxx 0.97
AKQJ9x.Jxx..K9xx xxx.xxx.AT8x.Qxx 8.KQ98.KJxx.AJT8 Txx.ATx.Q9xxx.xx 0.97
AKQJ9x.Jxx..K9xx Txx.Q.A8xxx.JTxx 8.AKTx.QJTx.Q8xx xxx.98xxx.K9xx.A 0.97
AKQJ9x.Jxx..K9xx 8xx.xxx.J8xxx.AT x.AT9x.AQ9x.QJ8x Txx.KQ8.KTxx.xxx 0.97
AKQJ9x.Jxx..K9xx Tx.K8xx.A9xxx.8x x.ATxx.KQJx.ATxx 8xxx.Q9.T8xx.QJx 0.96
AKQJ9x.Jxx..K9xx Tx.Axx.J8xxxx.Ax x.KQTx.KQ9x.QJ8x 8xxx.98x.ATx.Txx 0.96
